# Naive Bayes - Mushroom Dataset
Goal is to predict the class of mushrooms, given some features of the mushrooms. We will use Naive Bayes model for this classification.

### Load the Dataset

In [31]:
import numpy as np
import pandas as pd

In [32]:
df = pd.read_csv('../Datasets/Mushrooms/mushrooms.csv')
df.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [33]:
df.shape

(8124, 23)

### Encode the Categorical variable

In [34]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [35]:
le = LabelEncoder()
#Applies transformation in each col
ds = df.apply(func=le.fit_transform)

In [36]:
ds.head() #we have our encoded dataset

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [37]:
type(ds) #it is pandas df only

pandas.core.frame.DataFrame

In [38]:
data = ds.values
print(data.shape)
print(type(data))
print(data[:5,:]) # first 5 rows and all cols

(8124, 23)
<class 'numpy.ndarray'>
[[1 5 2 4 1 6 1 0 1 4 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 9 1 0 1 0 0 4 0 2 2 2 7 7 0 2 1 4 3 2 1]
 [0 0 2 8 1 3 1 0 0 5 0 2 2 2 7 7 0 2 1 4 3 2 3]
 [1 5 3 8 1 6 1 0 1 5 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 3 0 5 1 1 0 4 1 3 2 2 7 7 0 2 1 0 3 0 1]]


In [39]:
data_y = data[:,0]
data_x = data[:,1:]
# print(data_y)

### Break the data into train and test

In [40]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=0.2)

In [41]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(6499, 22) (6499,)
(1625, 22) (1625,)


In [42]:
np.unique(y_train) # find no of unique data in y_train

array([0, 1])

means there are only two classes

### Building our classifier

In [43]:
a = np.array([0,0,0,1,1,1,0,1])
np.sum(a == 1) # so this is how we calculate number of 1's in the array

4

In [44]:
def prior_prob(y_train, label):
    total_examples = y_train.shape[0]
    class_examples = np.sum(y_train==label)
    
    return (class_examples/float(total_examples))

In [45]:
# y = np.array([0,5,5,0,0,1,1,1,0,1])
# # print(len(y))
# prior_prob(y,1) #0.4 is true cuz we have 4 1's in 10 examples

In [46]:
def cond_prob(x_train,y_train,feature_col,feature_val,label):
    #finding num of examples with with that particular feature given that it is class label
    # for eg P(x=green/y=2)
    x_filtered = x_train[y_train==label] #first filter out the exmples having y has that label(y=2 for eg) means select only those rows having y as the label u want 
    numerator = np.sum(x_filtered[:,feature_col]==feature_val)
    denominator = np.sum(y_train==label)
    
    return numerator/float(denominator)
    
    

### Calculate Posterior Prob for each test example and make predictions

In [47]:
# np.unique(y_train)

In [48]:
def predict(x_train, y_train, xtest):
    """xtest is a single testing point, it has n number of features"""
    classes = np.unique(y_train)
    
    n_features = x_train.shape[1]
    post_probs = [] #List of prob for all classes given a single testing point
    #Compute posterior for each class
    for label in classes:
        #Post_c = likelihood*prior
        likelihood = 1.0
        for f in range(n_features):
            cond = cond_prob(x_train, y_train, f, xtest[f], label)
            likelihood *= cond
        
        prior = prior_prob(y_train, label)
        post = likelihood*prior
        post_probs.append(post)
        
    pred = np.argmax(post_probs)
    return pred
                

In [54]:
x_test[1]

array([2, 0, 3, 0, 2, 1, 0, 0, 3, 0, 1, 1, 1, 0, 4, 0, 2, 1, 2, 1, 4, 1])

In [55]:
output = predict(x_train, y_train, x_test[1])
print(output)
print(y_test[1])

1
1


In [60]:
def score(x_train, y_train, y_test):
    pred = []
    for i in range(x_test.shape[0]):
        pred_label = predict(x_train, y_train, x_test[i])
        pred.append(pred_label)
    pred = np.array(pred)
    
    accuracy = np.sum(pred==y_test)/y_test.shape[0]
    return accuracy*100
        

In [61]:
print(score(x_train, y_train, y_test))

99.93846153846154
